### annotation spacing 전후 volume값 구하여 비교

In [1]:
import os
import monai
import nibabel as nib
import numpy as np
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd
from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
)

# Enable deterministic training for reproducibility
monai.config.print_config()
monai.utils.set_determinism(seed=0)

def read_nifti_spacing_and_mask_count(nifti_file_path):
    try:
        # NIfTI 파일 열기
        img = nib.load(nifti_file_path)

        # spacing 정보 추출
        spacing = img.header.get_zooms()
        mask_voxel_count = np.count_nonzero(img.get_fdata()!=0)
        return mask_voxel_count, spacing[0], spacing[1], spacing[2]
    except Exception as e:
        print(f"Error reading NIfTI file: {str(e)}")
        return None

# Define a function to calculate the volume
def calculate_volume(mask_voxel_count, pixel_spacing_x, pixel_spacing_y, slice_gap):
#     return np.round((mask_voxel_count * pixel_spacing_x * pixel_spacing_y * slice_gap)/1000, 2)  ## mm3 to cc
    return (mask_voxel_count * pixel_spacing_x * pixel_spacing_y * slice_gap)/1000


directory_path = "/data/hanyang_Prostate/50_example/original/1_50_trim_20230130/Total volume/"
class_name = 1

if class_name == 1:
    datasets = "/data/hanyang_Prostate/50_example/trim/sl_data/centerCrop_350_350_200/dataset_fold_testjinjin.json"
    print("total_prostate test : dataset.json")
# if args.class_name == 2:
#     datasets = args.root_path + "/dataset_2_fold{}.json".format(args.fold)
#     print("transition zone train :dataset_2.json")
files = load_decathlon_datalist(datasets, True, "training")

transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAI"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(0.8, 0.8, 0.8),
            mode=("bilinear", "nearest"),
        ),
    ]
)

db = CacheDataset(
    data=files, transform=transforms, cache_num=6, cache_rate=1.0, num_workers=4
)

for _ in range(len(db)):
       
    mask_voxel_count2 = np.count_nonzero(db[_]['label'][0]!=0)
    volume2 = calculate_volume(mask_voxel_count2, transforms.transforms[-1].spacing_transform.pixdim[0], transforms.transforms[-1].spacing_transform.pixdim[1], transforms.transforms[-1].spacing_transform.pixdim[2])
    no = db[_]['label_meta_dict']['filename_or_obj'].split('/')[-1][:8]
    
    original_nifti_file_path=''
    # 디렉토리 탐색
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if no in file:
                # 검색한 문자열을 포함한 파일의 전체 경로 출력
                original_nifti_file_path = os.path.join(root, file)
#                 print("\t Original file name:", original_nifti_file_path)
                
    mask_voxel_count, pixel_spacing_x, pixel_spacing_y, slice_gap = read_nifti_spacing_and_mask_count(original_nifti_file_path)
    volume = calculate_volume(mask_voxel_count, pixel_spacing_x, pixel_spacing_y, slice_gap)
    
    print(f"No: {no}, Volume: {np.round(volume,1)} (cc), Transformed Volume: {np.round(volume2,1)} (cc)")
    print(f"\t Original Pixel Spacing (x): {np.round(float(pixel_spacing_x),3)}")
    print(f"\t Original Pixel Spacing (y): {np.round(float(pixel_spacing_y),3)}")
    print(f"\t Original Slice Gap: {np.round(float(slice_gap),3)}")
    print('------------------------------------------------------')
print('done')

/root/anaconda3/envs/prostate/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MONAI version: 1.2.0
Numpy version: 1.23.1
Pytorch version: 1.13.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: /root/anaconda3/envs/prostate/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.19.2
Pillow version: 6.2.1
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: NOT INSTALLED or UNKNOWN VERSION.
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

Fo

/root/anaconda3/envs/prostate/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


total_prostate test : dataset.json


Loading dataset: 100%|██████████| 6/6 [00:09<00:00,  1.57s/it]


No: 02953538, Volume: 35.9 (cc), Transformed Volume: 35.5 (cc)
	 Original Pixel Spacing (x): 0.782
	 Original Pixel Spacing (y): 0.782
	 Original Slice Gap: 0.8
------------------------------------------------------
No: 02953540, Volume: 27.1 (cc), Transformed Volume: 27.0 (cc)
	 Original Pixel Spacing (x): 0.816
	 Original Pixel Spacing (y): 0.816
	 Original Slice Gap: 0.8
------------------------------------------------------
No: 02953784, Volume: 33.8 (cc), Transformed Volume: 33.5 (cc)
	 Original Pixel Spacing (x): 0.782
	 Original Pixel Spacing (y): 0.782
	 Original Slice Gap: 0.8
------------------------------------------------------
No: 02953958, Volume: 26.5 (cc), Transformed Volume: 26.4 (cc)
	 Original Pixel Spacing (x): 0.801
	 Original Pixel Spacing (y): 0.801
	 Original Slice Gap: 0.8
------------------------------------------------------
No: 02954359, Volume: 22.4 (cc), Transformed Volume: 22.1 (cc)
	 Original Pixel Spacing (x): 0.782
	 Original Pixel Spacing (y): 0.782
	

No: 02573669, Volume: 32.5 (cc), Transformed Volume: 32.3 (cc)
	 Original Pixel Spacing (x): 0.782
	 Original Pixel Spacing (y): 0.782
	 Original Slice Gap: 0.8
------------------------------------------------------
No: 02603745, Volume: 10.1 (cc), Transformed Volume: 9.9 (cc)
	 Original Pixel Spacing (x): 0.782
	 Original Pixel Spacing (y): 0.782
	 Original Slice Gap: 0.8
------------------------------------------------------
No: 02610498, Volume: 21.1 (cc), Transformed Volume: 20.9 (cc)
	 Original Pixel Spacing (x): 0.782
	 Original Pixel Spacing (y): 0.782
	 Original Slice Gap: 0.8
------------------------------------------------------
No: 02744296, Volume: 27.0 (cc), Transformed Volume: 27.0 (cc)
	 Original Pixel Spacing (x): 0.866
	 Original Pixel Spacing (y): 0.866
	 Original Slice Gap: 0.8
------------------------------------------------------
No: 02786444, Volume: 19.1 (cc), Transformed Volume: 18.9 (cc)
	 Original Pixel Spacing (x): 0.782
	 Original Pixel Spacing (y): 0.782
	 

In [5]:
import numpy as np

dices=np.array([0.8727584053719873,0.886028415010999,0.845993061256093,0.8867125138100972,0.8800543297179617])*100
print("DICE")
print("Mean:", dices.mean(), "STD:", dices.std())

jaccs=np.array([0.7783564155370548,0.7963444527531813,0.7363816899794285,0.7987429792913189,0.7865555661466497])*100
print("Jacc")
print("Mean:", jaccs.mean(), "STD:", jaccs.std())

HDs=np.array([4.459498487141463,4.07205051820342,5.952619558402496,4.420096287594617,4.613919553666465])
print("HD")
print("Mean:", HDs.mean(), "STD:", HDs.std())

ASDs=np.array([1.7887478312346672,1.6129400814177521,2.244063198451727,1.7279977462609644,1.7353967994591124])
print("ASD")
print("Mean:", ASDs.mean(), "STD:", ASDs.std())

DICE
Mean: 87.43093450334275 STD: 1.502263217055699
Jacc
Mean: 77.92762207415267 STD: 2.2645330259819216
HD
Mean: 4.703636881001692 STD: 0.6491389811759107
ASD
Mean: 1.8218291313648447 STD: 0.21876265304846126


### fold별 각각의 prediction에 대한 volume 추정

In [7]:
mask_voxel_count, pixel_spacing_x, pixel_spacing_y, slice_gap = read_nifti_spacing_and_mask_count('/data/hanyang_Prostate/Prostate/prostate_1c_test_result/test/unet_10000_30000_fold1/0_00800131_gt.nii.gz')
volume = calculate_volume(mask_voxel_count, pixel_spacing_x, pixel_spacing_y, slice_gap)
print(pixel_spacing_x)
print(volume)

0.8
29.964289339508078


In [8]:
mask_voxel_count, pixel_spacing_x, pixel_spacing_y, slice_gap = read_nifti_spacing_and_mask_count('/data/hanyang_Prostate/Prostate/prostate_1c_test_result/test/unet_10000_30000_fold1/0_00800131_pred.nii.gz')
volume = calculate_volume(mask_voxel_count, pixel_spacing_x, pixel_spacing_y, slice_gap)
print(pixel_spacing_x)
print(volume)

0.8
29.287425309249894


In [10]:
mask_voxel_count, pixel_spacing_x, pixel_spacing_y, slice_gap = read_nifti_spacing_and_mask_count('/data/hanyang_Prostate/50_example/original/1_50_trim_20230130/Total volume/3D_00800131_20220118_5_1.nii.gz')
volume = calculate_volume(mask_voxel_count, pixel_spacing_x, pixel_spacing_y, slice_gap)
print(pixel_spacing_x)
print(volume)

0.782
30.270930331711966
